# Experiment: _Relabel graphs_

Here, we relabel graph nodes which occur only non-frequent in the dataset. Eg. label 'something' occurs only once in the training set, so relabel with another label that is similar to it and occurs more often.

In [ ]:
from notebook_prelude import *

In [ ]:
df = results_helper.get_experiments_by_names(['experiment_relabel'])
df_ = results_helper.get_results()
df = df.append(df_).fillna('-')
print('# Results: {}'.format(len(df)))

In [ ]:
df.columns

In [ ]:
df[(df.combined == False) & (df.type != TYPE_COOCCURRENCE) ].groupby(['dataset', 'type', 'graph_preprocessing', 'graph__phi_picker__use_zeroth']).mean_test_f1_macro.max().to_frame()

In [ ]:
df.features__fast_wl_pipeline__feature_extraction__graph_preprocessing.unique()